In [1]:
import json
import shutil
import os
import requests

In [2]:
def get_json_data(lat, lng):
    api_url = 'https://mtf-sat.synvinkel.org/timeseries?apikey=biscayabukten&fbclid=IwAR0woGRqFpiBML0h03XRs9Uz5ag3Vw834foaHnW4oCyRbRxjVyU8mmKOILE'
    #always whole 2018
    paramters  =' &lng=' + str(lng) + '&lat=' + str(lat) + '8&startDate=2018-01-01&endDate=2018-12-30'
    req = requests.get(api_url + paramters)
    if req.status_code == 200:
        req_data = req.content
        return json.loads(req_data)
    else:
        return None

In [3]:
json_data = get_json_data(lng=106.20,lat=53.98)
json_data

{'location': {'lng': '106.2', 'lat': '53.988'},
 'images': [{'bands': {'B1': 7802.01,
    'B10': 1250.28,
    'B11': 2957.63,
    'B12': 2939.7,
    'B2': 7838.17,
    'B3': 7012.38,
    'B4': 8195.28,
    'B5': 8662.8,
    'B6': 8970.38,
    'B7': 9085.81,
    'B8': 8903.6,
    'B8A': 9199.22,
    'B9': 5884.63},
   'cloudcover': 61.3645,
   'date': '2018-01-06',
   'time': 1515211892460,
   'url': 'https://mtf-sat.synvinkel.org/image/106.2/53.988/20180106T041129_20180106T041132_T48UWE-7d0.png',
   'rawUrl': 'https://mtf-sat.synvinkel.org/image/106.2/53.988/20180106T041129_20180106T041132_T48UWE-7d0.zip'},
  {'bands': {'B1': 5486.31,
    'B10': 226.19,
    'B11': 1387.25,
    'B12': 1308.47,
    'B2': 5048.61,
    'B3': 4232.46,
    'B4': 4519.2,
    'B5': 4624.67,
    'B6': 4715.92,
    'B7': 4750.47,
    'B8': 4451.83,
    'B8A': 4707.15,
    'B9': 2065.19},
   'cloudcover': 99.9999,
   'date': '2018-01-11',
   'time': 1515643885460,
   'url': 'https://mtf-sat.synvinkel.org/image/10

In [6]:
def toString(long, lat):
    return "lng:" + long + "lat:" + lat

def download_files(rel_dir, json_req):
    if not os.path.exists(rel_dir):
        os.makedirs(rel_dir)

    name = toString(json_data["location"]["lng"], json_data["location"]["lat"]) + "/"

    if not os.path.exists(rel_dir + name):
        os.makedirs(rel_dir + name)
    
    for image in json_data["images"]:
        response = requests.get(image["url"], stream=True)
        file_name = rel_dir + name + "time:" + str(image["time"]) + ".png"
        with open(file_name, 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
        del response

In [7]:
download_files("./downloads/", json_data)

## Generate versioncontrollable artifacts

In [ ]:
!jupyter nbconvert --output-dir='./target' --to python get_data_scripts.ipynb

In [ ]:
!jupyter nbconvert --output-dir='./target' --to html get_data_scripts.ipynb